In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

24/06/25 19:06:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
Employees = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Employees") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

### Query -1 

In [12]:
layer_1_df = Employees\
    .filter((col("manager_id")==1) & (col("employee_id") != col("manager_id")))\
    .select("employee_id")
layer_1 = [row.employee_id for row in layer_1_df.collect()]
print(layer_1)

layer_2_df = Employees\
    .filter(col("manager_id").isin(layer_1))\
    .select("employee_id")
layer_2 = [row.employee_id for row in layer_2_df.collect()]
print(layer_2)

layer_3_df = Employees\
    .filter(col("manager_id").isin(layer_2))\
    .select("employee_id")
layer_3 = [row.employee_id for row in layer_3_df.collect()]
print(layer_3)

layer_1_df\
    .unionAll(layer_2_df)\
    .unionAll(layer_3_df)\
    .show()


[2, 77]
[4]
[7]
+-----------+
|employee_id|
+-----------+
|          2|
|         77|
|          4|
|          7|
+-----------+

